# Shekel

Consider the Shekel function
 \begin{equation}
 \label{eq:model}
f(x)=-\sum_{p=1}^{P} ((x-x_{0p})^T A_p^T \Sigma_p A_p (x-x_{0p}) + b_p)^{-1},
\end{equation}
where $x \in \mathfrak{X}=[0,10]^d$, $P$ is the number of minima; $A_p$ is a random rotation
matrix of size $d\times d$; $\Sigma_p$ is a diagonal positive
definite matrix of size $d\times d$ with smallest and largest
eigenvalues $\lambda_{min}$ and $\lambda_{max}$ respectively;
$x_{0p} \in \mathfrak{X}$, $b_p$ is a constant and $p=1,...,P$.

The gradient of the Shekel function is

\begin{equation}
\nabla f(x)= \sum_{p=1}^{P}\frac{2A_p^T \Sigma_p A_p(x -x_{0p})}{((x -x_{0p})^T A_p^T \Sigma_p A_p (x-x_{0p}) + b_p)^2}.
\end{equation}



# Import libraries

In [60]:
import numpy as np 
from numpy import linalg as LA
import pandas as pd
import matplotlib.pyplot as plt; plt.rcdefaults()
import textwrap

import metod_alg as mt
from metod_alg import metod_algorithm_functions as mt_alg
from metod_alg import objective_functions as mt_obj
import tqdm
import seaborn as sns
import SALib
from SALib.sample import sobol_sequence

# Results

For all results $d=4$ and $N=100$.

In [68]:
d = 4
p = 10
beta_list = [0.1]
m_list = [1]
    
avg_grad = np.zeros((len(m_list) * len(beta_list), 100, 500))

total_no_local_minimizers_metod = np.zeros((len(m_list), len(beta_list), 100))
time_taken_metod = np.zeros((len(m_list), len(beta_list), 100))
extra_descents_metod = np.zeros((len(m_list), len(beta_list), 100))
func_val_metod = np.zeros((len(m_list), len(beta_list), 100))

df_mult = pd.read_csv('shekel_sd_metod_beta_0.1_m=1_d=4_p=%s_random_100.csv' % (p))
total_no_local_minimizers_mult = np.array(df_mult['number_minimizers_per_func_multistart'])
time_taken_mult = np.array(df_mult['time_multistart'])
func_val_mult = np.array(df_mult['min_func_val_multistart'])
test = np.array(df_mult['number_minimizers_per_func_metod'])

# index_all = 0
# index_m = 0
# for m in m_list:
#     index_beta = 0
#     for beta in beta_list:
#         df_metod = pd.read_csv('shekel_metod_beta_%s_m=%s_d=4_p=%s_random_100.csv'% (beta, m, p))
#         total_no_local_minimizers_metod[index_m, index_beta] = np.array(df_metod['number_minimizers_per_func_metod'])
#         time_taken_metod[index_m, index_beta] = np.array(df_metod['time_metod'])
#         func_val_metod[index_m, index_beta] = np.array(df_metod['min_func_val_metod'])
#         if beta == 0.1 and m == 1:
#             assert(np.all(total_no_local_minimizers_metod[index_m, index_beta] == test))
#         extra_descents_metod[index_m, index_beta] = np.array(df_metod['number_extra_descents_per_func_metod'])

#         avg_grad[index_all] = np.genfromtxt('shekel_grad_norm_beta_%s_m=%s_d=4_p=%s_random_100.csv'% (beta, m, p), delimiter=',')
#         index_beta += 1
#         index_all += 1
#     index_m += 1

In [74]:
np.sum(total_no_local_minimizers_mult == test)

69

In [71]:
total_no_local_minimizers_mult

array([10.,  8., 10.,  9.,  9., 10.,  9., 10.,  8., 10., 10., 10.,  9.,
        8., 10., 10.,  9., 10., 10.,  9., 10., 10.,  9.,  9.,  9., 10.,
        9., 10., 10., 10.,  7., 10., 10.,  9.,  9., 10.,  9.,  8.,  9.,
        8.,  9.,  9.,  9.,  9.,  9.,  9., 10.,  9., 10., 10.,  9.,  8.,
        9., 10.,  9.,  9., 10.,  9., 10.,  9.,  9., 10., 10., 10., 10.,
       10.,  9., 10.,  9.,  9., 10.,  9.,  8., 10.,  9., 10.,  9.,  9.,
        7.,  8., 10., 10., 10.,  9.,  9., 10., 10., 10., 10.,  9.,  9.,
       10., 10.,  9.,  9., 10., 10.,  9., 10.,  9.])

In [72]:
test

array([10.,  8., 10.,  9.,  9., 10.,  9., 10.,  8.,  9., 10., 10.,  9.,
        8., 10.,  9.,  9., 10., 10.,  9.,  8., 10.,  9.,  9.,  8.,  9.,
        9., 10.,  9., 10.,  7.,  9., 10.,  9.,  8., 10.,  8.,  8.,  8.,
        8.,  9.,  8.,  9.,  8.,  9.,  9.,  9.,  9., 10.,  9.,  8.,  7.,
        9., 10.,  8.,  9., 10.,  8., 10.,  9.,  9., 10., 10., 10., 10.,
       10.,  9.,  9.,  9.,  8.,  9.,  9.,  8., 10.,  8., 10.,  9.,  9.,
        6.,  8., 10.,  9., 10.,  8.,  9., 10., 10.,  9., 10.,  8.,  9.,
       10., 10.,  8.,  8.,  9., 10.,  9.,  9.,  9.])

In [73]:
np.array(df_mult['number_extra_descents_per_func_metod'])

array([38., 39., 45., 37., 31., 46., 35., 44., 51., 42., 45., 46., 48.,
       40., 45., 37., 35., 33., 41., 33., 48., 40., 38., 36., 40., 51.,
       28., 42., 66., 44., 47., 41., 50., 33., 54., 41., 58., 33., 28.,
       41., 46., 49., 38., 40., 50., 43., 46., 28., 32., 50., 44., 44.,
       36., 40., 51., 35., 36., 48., 44., 35., 48., 48., 34., 43., 43.,
       53., 43., 34., 50., 35., 37., 51., 32., 43., 39., 27., 40., 47.,
       41., 40., 35., 43., 40., 32., 46., 46., 34., 36., 58., 26., 38.,
       44., 50., 43., 39., 48., 45., 46., 35., 36.])

In [63]:
total_no_local_minimizers_mult == test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [58]:
test

array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 4., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 4., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [59]:
np.array(df_mult['number_extra_descents_per_func_metod'])

array([53., 43., 57., 54., 40., 47., 39., 45., 47., 48., 42., 53., 54.,
       36., 53., 47., 56., 50., 61., 56., 59., 43., 55., 41., 42., 51.,
       35., 48., 44., 49., 48., 49., 43., 61., 41., 62., 54., 43., 42.,
       43., 48., 57., 52., 57., 50., 50., 50., 40., 51., 52., 41., 54.,
       40., 47., 54., 48., 52., 50., 45., 48., 54., 46., 43., 60., 53.,
       49., 54., 37., 40., 51., 41., 52., 54., 44., 55., 46., 42., 46.,
       53., 47., 56., 50., 57., 51., 52., 52., 61., 53., 49., 45., 52.,
       67., 49., 54., 51., 49., 50., 45., 41., 59.])

In [10]:
np.array(df_mult['time_metod']) / np.array(df_mult['time_multistart'])

array([0.68077547, 0.54100868, 0.81001014, 0.60411696, 0.56141288,
       0.57418507, 0.55506379, 0.58892296, 0.57483125, 0.65150728,
       0.53524271, 0.63377536, 0.68588316, 0.48777765, 0.52281426,
       0.57990956, 0.76996363, 0.64465454, 0.74097579, 0.66282351,
       0.74520251, 0.57268519, 0.68132674, 0.52998083, 0.51772478,
       0.63996203, 0.45365273, 0.57047358, 0.58376671, 0.61288788,
       0.53746507, 0.62594968, 0.56124554, 0.77811545, 0.51286002,
       0.69859373, 0.59711192, 0.5867975 , 0.53670184, 0.51135847,
       0.61625616, 0.74475198, 0.66465686, 0.69495758, 0.63443534,
       0.6074395 , 0.61962324, 0.49576441, 0.64144662, 0.63432316,
       0.5263817 , 0.65065603, 0.50227373, 0.53904797, 0.67693728,
       0.59545927, 0.66597242, 0.64507652, 0.57283838, 0.60157461,
       0.67409233, 0.57343476, 0.55924521, 0.71783005, 0.62175109,
       0.61238687, 0.65059578, 0.49421619, 0.52597394, 0.62916075,
       0.49711224, 0.68961382, 0.67437768, 0.56563648, 0.67981

In [ ]:
total_no_minimizers_prop = total_no_local_minimizers_metod / total_no_local_minimizers_mult

In [ ]:
time_taken_prop = time_taken_metod / time_taken_mult

In [ ]:
for j in range(len(beta_list) * len(m_list)):
    for k in range(j+1, len(beta_list) * len(m_list)):
            assert(np.all(np.round(avg_grad[k], 5) == np.round(avg_grad[j], 5)))

In [ ]:
same_global_min = np.zeros((len(m_list), len(beta_list)))
same_minimizer = np.zeros((len(m_list), len(beta_list)))
mean_excessive_descents = np.zeros((len(m_list), len(beta_list)))
for i in range(len(m_list)):
    for j in range(len(beta_list)):
        assert(np.all(total_no_local_minimizers_mult >=  total_no_local_minimizers_metod[i, j]))
        same_global_min[i, j] = np.where(np.round(func_val_mult, 2) == np.round(func_val_metod[i, j], 2))[0].shape[0]
        same_minimizer[i, j] = np.sum(total_no_local_minimizers_metod[i, j]) / np.sum(total_no_local_minimizers_mult)
        mean_excessive_descents[i, j] = np.mean(extra_descents_metod[i, j])

In [ ]:
same_global_min

In [ ]:
same_minimizer

In [12]:
number_its = np.genfromtxt('shekel_no_its_mult_beta_0.1_m=1_d=4_p=%s_random_100.csv' % (p), delimiter=',')
store_mean_its = np.zeros((100))
store_min_its = np.zeros((100))
store_mean_norm_grad = np.zeros((100))
for j in range(100):
    store_mean_its[j] = np.mean(number_its[j])
    store_min_its[j] = np.min(number_its[j])
#     store_mean_norm_grad[j] = 1 / np.mean(avg_grad[0][j])

In [13]:
store_mean_its

array([53.82, 50.81, 52.35, 49.35, 47.21, 53.12, 49.21, 50.46, 52.09,
       50.77, 54.52, 48.65, 48.91, 51.79, 47.39, 51.89, 54.9 , 49.57,
       50.02, 48.59, 44.85, 52.19, 50.16, 51.21, 49.36, 49.81, 51.4 ,
       48.73, 53.58, 46.34, 53.31, 53.2 , 53.11, 45.26, 52.42, 51.47,
       50.45, 48.22, 47.77, 51.83, 49.95, 45.15, 51.37, 49.81, 50.97,
       49.77, 49.07, 49.62, 49.93, 48.11, 52.05, 47.32, 45.32, 51.03,
       47.49, 53.49, 45.35, 51.6 , 53.99, 49.83, 52.91, 53.9 , 50.71,
       49.68, 51.43, 50.87, 51.14, 49.31, 49.16, 52.  , 50.18, 43.62,
       49.34, 49.65, 50.84, 51.5 , 49.17, 50.26, 44.64, 50.81, 54.29,
       51.38, 52.36, 47.83, 53.01, 49.22, 50.41, 50.77, 52.47, 48.21,
       49.01, 51.63, 52.59, 54.14, 54.73, 51.53, 50.95, 49.25, 52.22,
       54.18])

In [14]:
store_min_its

array([30., 27., 27., 26., 18., 28., 24., 27., 35., 32., 37., 28., 25.,
       14., 24., 26., 33., 31., 23., 27., 19., 31., 29., 23., 17., 29.,
       31., 22., 35., 19., 30., 36., 30., 20., 28., 24., 24., 33., 21.,
       20., 32., 19., 27., 30., 33., 25., 21., 20., 28., 25., 29., 16.,
       25., 27., 29., 30., 17., 33., 34., 25., 33., 26., 36., 24., 20.,
       26., 30., 25., 29., 29., 29., 19., 27., 27., 32., 27., 29., 28.,
       23., 26., 38., 15., 26., 20., 33., 25., 35., 26., 27., 31., 18.,
       31., 31., 23., 32., 34., 29., 26., 26., 25.])

In [ ]:
store_mean_norm_grad

In [ ]:
def set_box_color(bp, color):
    """Set colour for boxplot."""
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)
    
def create_boxplots_ratio(arr1, arr2, labels, ticks):
    plt.figure(figsize=(7, 5))
    
    max_num = max(np.max(arr1), np.max(arr2))
    plt.ylim(0, max_num + 0.1)
    bpl = plt.boxplot(arr1.T,
                      positions=np.array(range(len(arr1)))*2.0-0.4)
    bpr = plt.boxplot(arr2.T,
                      positions=np.array(range(len(arr2)))*2.0+0.4)
    set_box_color(bpl, 'green')
    set_box_color(bpr, 'navy')
    plt.plot([], c='green', label=labels[0])
    plt.plot([], c='navy', label=labels[1])
    plt.legend(bbox_to_anchor=(0.99, 1.025), loc='upper left',
               prop={'size': 15})
    plt.xlabel(r'$\beta$', size=14)
    plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks, size=15)
    plt.yticks(fontsize=14)
    plt.tight_layout()

# Total number of local minima

In [ ]:
ticks = [beta_list[0],
          beta_list[1],
          beta_list[2]]
labels = [r'$M =$ %s' % (m_list[0]),
        r'$M =$ %s' % (m_list[1])]

In [ ]:
create_boxplots_ratio(total_no_minimizers_prop, total_no_minimizers_prop_diff, labels, ticks)

# Efficiency

In [ ]:
create_boxplots_ratio(time_taken_prop[0], time_taken_prop[1], labels, ticks)

# Excessive descents

In [ ]:
create_boxplots_ratio(extra_descents_metod[0], extra_descents_metod[1], labels, ticks)